In [7]:
import json
import logging

import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

In [8]:
class BedrockRuntimeWrapper:
    def __init__(self, bedrock_runtime_client):
        """
        :param bedrock_runtime_client: A low-level client representing Amazon Bedrock Runtime.
                                       Describes the API operations for running inference using
                                       Bedrock models.
        """
        self.bedrock_runtime_client = bedrock_runtime_client
        
    def invoke_mistral_7b(self, prompt):
        """
        Invokes the Mistral 7B model to run an inference using the input
        provided in the request body.

        :param prompt: The prompt that you want Mistral to complete.
        :return: List of inference responses from the model.
        """

        try:
            # Mistral instruct models provide optimal results when
            # embedding the prompt into the following template:
            instruction = f"<s>[INST] {prompt} [/INST]"

            model_id = "mistral.mistral-7b-instruct-v0:2"

            body = {
                "prompt": instruction,
                "max_tokens": 200,
                "temperature": 0.5,
            }

            response = self.bedrock_runtime_client.invoke_model(
                modelId=model_id, body=json.dumps(body)
            )

            response_body = json.loads(response["body"].read())
            outputs = response_body.get("outputs")

            completions = [output["text"] for output in outputs]

            return completions

        except ClientError:
            logger.error("Couldn't invoke Mistral 7B")
            raise

In [9]:
def invoke(wrapper, model_id, prompt):
    print("-" * 88)
    print(f"Invoking: {model_id}")
    print("Prompt: " + prompt)

    try:
        if model_id == "anthropic.claude-v2":
            completion = wrapper.invoke_claude(prompt)
            print("Completion: " + completion)

        elif model_id == "ai21.j2-mid-v1":
            completion = wrapper.invoke_jurassic2(prompt)
            print("Completion: " + completion)

        elif model_id == "meta.llama2-13b-chat-v1":
            completion = wrapper.invoke_llama2(prompt)
            print("Completion: " + completion)

        elif model_id == "mistral.mistral-7b-instruct-v0:2":
            completions = wrapper.invoke_mistral_7b(prompt)
            for completion in completions:
                print("Completion: " + completion)

        elif model_id == "mistral.mixtral-8x7b-instruct-v0:1":
            completions = wrapper.invoke_mixtral_8x7b(prompt)
            for completion in completions:
                print("Completion: " + completion)
        
        return completions

    except ClientError:
        logger.exception("Couldn't invoke model %s", model_id)
        raise

In [10]:
client = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")
wrapper = BedrockRuntimeWrapper(client)

text_generation_prompt  = "Verify the following statement: Donald Trump is 60 years old"
invoke(wrapper, "mistral.mistral-7b-instruct-v0:2", text_generation_prompt)

----------------------------------------------------------------------------------------
Invoking: mistral.mistral-7b-instruct-v0:2
Prompt: Verify the following statement: Donald Trump is 60 years old
Completion:  I cannot directly verify the statement as I do not have access to real-time or personal information about individuals. However, I can tell you that Donald Trump was born on June 14, 1946. To calculate his age, you can subtract his birth year from the current year. As of now, the current year is 2022, so the calculation would be: 2022 - 1946 = 76. Therefore, Donald Trump would be 76 years old. The statement that he is 60 years old is incorrect.
